In [1]:
# ============================================================
# Notebook setup: run this before everything
# ============================================================

%load_ext autoreload
%autoreload 2

# Control figure size
interactive_figures = False
if interactive_figures:
    # Normal behavior
    %matplotlib widget
    figsize=(9, 3)
else:
    # PDF export behavior
    figsize=(14, 5)

import seaborn as sns
import matplotlib.pyplot as plt
from util import util
#from scipy.integrate import odeint
import numpy as np
import pandas as pd
import os
import json
#from skopt.space import Space
#from eml.net.reader import keras_reader
from codecarbon import EmissionsTracker
from sklearn.tree import DecisionTreeRegressor
from argparse import ArgumentParser

# Sustainable Hardware Dimensioning (AIITI Exam)

In this work, we explore the problem of finding the the best Hardware architecture and its Dimensioning for AI algorithms, while respecting constraints in terms of carbon emissions. An existing tool, named [HADA](https://www.sciencedirect.com/science/article/abs/pii/S0950705122005974) (HArdware Dimensioning of AI Algorithms)  has focused on the task of Hardware Dimensioning with constraints on budget, runtime and solution quality. We aim at extending this approach by also considering constraints on carbon emissions resulting from computations, and we name this problem Sustainable Hardware Dimensioning.

## Methodology

The HADA approach is based on the [Empirical Model Learning (EML)](https://www.sciencedirect.com/science/article/pii/S0004370216000126) paradigm, which integrates Machine Learning (ML) models into an optimisation problem. Broadly speaking, EML deals with solving declarative optimisation models with a complex component $h$, which represents the relation between variables which can be acted upon $x$ (the decision variables) and the *observables* $y$ related to the system considered; the function $h(x) = y$ describes this relationships. As the $h(x)$ is complex, we cannot optimise directly over it. Hence, we exploit empirical knowledge to build a surrogate model $h_\theta(x)$ learned from data, where $\theta$ is the parameter vector.

HADA is then constituted of three main phases:

1. **Data Set Collection** (benchmarking phase): an initial phase to collect the data set by running multiple times the target algorithms, under different configurations of algorithm hyperparameters and Hardware;
2. **Surrogate Model Creation**: once a training set is available, a set of ML models is then trained on such data. These are the surrogate models $h_{\theta}(x)$, which are then encoded in the optimisation problem as a set of variables and constraints following the EML paradigm;
3. **Optimisation**: post the user-defined constraints and objective function on top of the combinatorial structure formed by the encoded ML models and the domain-knowledge constraints, and finally solve the optimisation model.

The original HADA work focuses on two stochastic algorithms, i.e., anticipate and contingency [[3]](https://www.ijcai.org/proceedings/2019/150) from the energy management system domain. The two algorithms calculate the amount of energy that must be produced by the energy system to meet the required load, minimising the total energy cost over the daily time horizon and by taking into account the uncertainty. 

### What we will do in this work

The code implementing the two algorithms as well as the HADA tool has already been developed. In particular there is a version of HADA implemented as a Flask web application, that given the Data Sets of the target algorithms, it provides a user interface to select the algorithm, specify the obective and the constraints. Then, it creates the ML models under the hood, embeds them in the optimisation model, adds the user-defined constraints and solves the model for the required objective, namely the phases (2) and (3) described above.



Then, the focus of the current work is:

* Measure the carbon emissions of the target algorithms, namely anticipate and contingency
* Collect new datasets running the algorithms over a set of instances with different configurations and on different HW platforms;
* Update the existing HADA code for accomodating carbon emissions.

## Measuring Carbon Emissions

The **Carbon Emissions** are expressed in terms of carbon dioxide equivalent $\mathsf{CO_2e}$, and summarise the global warming effect of the Greenhouse Gases (GHG) emitted in the determined timeframe. To measure the Carbon Emissions we rely on [codecarbon](https://mlco2.github.io/codecarbon/index.html), which is a python package offering useful tools for tracking the emissions resulting from executing code execution. Codecarbon computes the Carbon Emissions ($\mathsf{C}$) as:

\begin{equation}
    \mathsf{C} = \mathsf{E} \times \mathsf{CI}
\end{equation}

Where:
* $\mathsf{E}$ is the Energy consumed by the computational infrastructure, quantified as kilowatt-hours (kWh).
* $\mathsf{CI}$ is the **Carbon Intensity** of the electricity consumed for computation, quantified as g of $\mathsf{CO_2e}$ emitted per kilowatt-hour (kWh) of electricity.

Codecarbon directly measures the energy consumption of the CPU, GPU and RAM on which the code is executed at intervals of 15 sec (by default, but it can also be modified). For example, let's track the emissions of running the ANTICIPATE algorithm. Let's say we would like to solve instance 5 with 4 scenarios:

In [2]:
scenarios = 4
instance = 5
project_name = f"anticipate-ins-{instance}-ns-{scenarios}"
output_dir = '../data/'

# Codecarbon emission tracker
tracker = EmissionsTracker(project_name=project_name, 
                           log_level='ERROR', 
                           output_dir=output_dir)

# Use the emission tracker as a context manager
with tracker as t:
    sol_cost, run_final, mem_final = util.online_ant(scenarios=scenarios, instance=instance, file='InstancesTest.csv')

print(f"The solution cost (in keuro) is: {sol_cost:.2f}")
print(f"The runtime (in sec) is: {run_final:.2f}")
print(f"Avg memory used (in MB) is: {mem_final:.2f}")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2512647
Academic license 2512647 - for non-commercial use only - registered to en___@studio.unibo.it
The solution cost (in keuro) is: 370.29
The runtime (in sec) is: 8.64
Avg memory used (in MB) is: 225.84


This tracks the emissions of the `online_ant` function, which executes the ANTICIPATE algorithm, by using the codecarbon `EmissionTracker` as a context manager. By default, codecarbon saves the tracking data to a .csv file, named `emissions.csv`. Let's take a look:

In [3]:
emissions = util.display_emissions_data()
display(emissions)

,timestamp,project_name,duration,emissions,emissions_rate,cpu_power,ram_power,cpu_energy,ram_energy,energy_consumed,...,region,os,python_version,codecarbon_version,cpu_count,cpu_model,longitude,latitude,ram_total_size,tracking_mode
0,2024-11-22T10:03:52,anticipate-ins-5-ns-4,15.26002,0.000063,0.000004,42.5,1.433251,0.00018,0.000006,0.000186,...,emilia-romagna,Linux-6.10.4-linuxkit-x86_64-with-glibc2.31,3.9.20,2.3.2,8,Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz,12.0658,44.131,3.822002,machine


As we can see, Codecarbon keeps track of a series of metrics. For this project, the metrics we decided to include in the training set to generate are the following:

* `emissions`: the total emissions of CO2eq (kg);
* `emission_rate`: the amount of CO2eq emissions per second (kg/s);
* `memPeak(MB)`: the maximum amount of memory required by the algorithm (in MB);
* `cpu_energy`: the energy consumed by the cpu;
* `ram_energy`: the energy consumed by the ram;
* `tot_energy`: the total energy consumed;
* `country`, `region`: the country and region where the computation took place;
* `cpu_count`: the number of cores.

## Dataset Collection

### Input data

The input data is a set of 30 different instance realisations, each one representing one daily time horizon. For each day, we have **Load**, which is a 96-valued vector of the load observations sampled at each interval (every 15 minutes over the course of a day), and **PV**, a 96-valued vector representing the observations of available Photovoltaic energy production. Here we can see an example of the first two instances:

In [4]:
# Data exploration: look at the input data
data = util.display_instances_data()
display(data)

,PV(kW),Load(kW)
0,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 10.4 20.4 29.8 25.9 37.6 54.9 58.6 114.2 122.2 241.2 288.2 265. 293.7 387.4 563. 560.9 447.7 569.6 555.8 307.2 693.7 718.5 767.9 664.5 724. 651.2 692. 650.5 673.5 378.8 720.2 654.4 417.8 546. 403.8 715. 517.1 581.9 669.2 629.5 672.7 640.5 653.5 655.6 655.1 416.2 318.9 569.6 523.5 511. 485.5 314.1 265.8 153.8 93. 110.5 148. 275.5 223.1 88.2 71. 58.8 31.8 32.8 12.9 21.3 23.9 20.4 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ],[106.9 130.3 140.2 98.7 85. 127.4 132. 115.3 130.3 89.4 84.3 90.9 104.7 97. 97.2 95.3 81.6 107.3 82. 102.6 108.9 81.2 108. 83.7 149.4 172.7 181.9 202.5 259.5 330.1 375.8 365.2 307.7 442.7 299.7 239.9 287.7 367.2 305.9 256. 322.4 399.1 276.6 258.9 281.1 266.7 306.2 289.1 296.5 346.8 235.7 236.3 240.1 228.2 277.4 251.5 274.8 291.5 325.6 363.3 317. 331.7 293. 346.5 385.5 384.1 351.7 395.3 596.6 443.5 585.6 561.8 491.7 616.5 562.3 438.4 432.3 563.6 628.8 448.1 581.8 531.6 551.6 433.3 701.6 635.7 580.8 544.6 561.5 656.2 408.2 414.3 387.8 353.7 311.2 230.4]
1,[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 14.4 20.4 29.8 25.9 42. 56.7 60.7 114.2 133.5 253.6 288.2 265. 293.7 387.4 519.1 560.9 457.3 532.3 532.6 269.8 723.3 718.5 767.9 664.5 637.1 651.2 655.9 657.4 770.6 378.8 698.6 654.4 417.8 546. 403.8 647.2 517.1 581.9 669.2 629.5 672.7 640.5 639.1 659.8 642.2 416.2 318.9 569.6 509.2 491.1 461. 314.1 246.7 151.2 102.4 110.5 148. 275.5 233.9 88.2 71. 64.1 31.8 31.6 12.9 25.3 20. 20.4 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ],[106.9 130.3 128.8 98.7 85. 126.2 109.2 144.6 130.3 92.9 84.3 96.9 121.3 73.9 109. 95.3 90.1 95.8 82. 112.4 108.9 103.1 108. 96.5 149.4 174.5 181.9 202.5 242. 330.1 375.8 399.2 428.6 325.8 256.3 334.8 306.5 367.2 286. 264.8 322.4 399.1 276.6 258.9 281.1 337.4 329.7 289.1 341.8 300.4 235.7 236.3 240.1 228.2 262.2 251.5 274.8 291.5 325.6 363.3 397.9 331.7 305.6 346.5 385.5 458.4 351.7 395.3 456.4 443.5 553.9 561.8 491.7 542.3 562.3 438.4 432.3 508.6 536. 448.1 435.7 538.2 436.5 433.3 701.6 682.1 580.8 544.6 561.5 655.9 533.5 414.3 399.7 353.7 311.2 230.4]


Since the objective of the two algorithms is to minimize the total energy cost, as input file we also have the grid electricity price at each our of the day

In [5]:
data = util.display_prices_data()
display(data)

,Time,Price
0,0,46.99
1,1,42.81
2,2,39.83
3,3,38.02
4,4,37.00


### Benchmarking

For the benchmarking phase, the ANTICIPATE and CONTINGECY algorithms were run on each instance 100 times, each time considering a different number of the configurable parameter (from 1 to 100 traces/scenarios). This value is taken directly from the HADA paper, according to which running the algorithms on each instance 100 times sufficiently explores the parameter space. 

Since for the HADA approach is recommended to collect data relative to different Hardware configurations, i executed the benchmarking phase on my personal laptop (A 2019 MacBook Pro), and on [Leonardo](https://leonardo-supercomputer.cineca.eu/), an HPC System hosted by CINECA. Then, the training set of each algorithm will be of 6,000 records (100 runs x 30 instances x 2 Hardware configurations).

Since running both algorithms 100 times over each instance with a different hyperparameter value is a time consuming (and repetitive) task, it would be useful to have some automatioin. Therefore i have used a python script that allows you to specify the instances and the hyperparameter values to use as command line arguments

**Benchmark on Leonardo**: The process to execute ANTICIPATE and CONTINGENCY on Leonardo presented some challenges. Both algorithms use the [Gurobi solver](https://www.gurobi.com/), and the process of figuring out how to use it was not straightforward, since it was not possible to install a license directly on the compute node where the algorithms would run. Therefore, this phase took quite some time to figure out the right way to proceed, and required the cooperation of CINECA staff and Gurobi's assistance. In the end, the solution was to install an *Academic Floating License* on the CINECA License Server, and use a Client License to connect to the server where the license resides once the solver is run.

After the benchmark phase, the collected dataset will look something like this:

In [6]:
filename = 'contingency_mbp19.csv'
benchmark_data = util.display_benchmark_data(filename)
display(benchmark_data)

,nTraces,sol(keuro),time(sec),memAvg(MB),memPeak(MB),CO2e(kg),CO2eRate(kg/s),cpuEnergy(kW),ramEnergy(kW),totEnergy(kW),country,region,cpuCount
0,1,369.19,7.75,162.51,170.40,7.90e-06,4.31e-06,2.17e-05,1.53e-06,2.32e-05,Italy,emilia-romagna,8
1,2,390.20,10.93,165.13,173.15,3.65e-06,4.31e-06,9.99e-06,7.05e-07,1.07e-05,Italy,emilia-romagna,8
2,3,374.38,9.40,168.86,177.57,2.66e-06,4.31e-06,7.30e-06,5.15e-07,7.82e-06,Italy,emilia-romagna,8
3,4,332.00,10.24,172.66,181.44,2.78e-06,4.31e-06,7.63e-06,5.39e-07,8.17e-06,Italy,emilia-romagna,8
4,5,333.78,11.16,176.31,185.48,2.97e-06,4.31e-06,8.13e-06,5.74e-07,8.71e-06,Italy,emilia-romagna,8
5,6,333.63,12.13,180.02,189.12,3.14e-06,4.31e-06,8.60e-06,6.07e-07,9.21e-06,Italy,emilia-romagna,8
6,7,341.48,13.51,184.00,193.00,3.49e-06,4.31e-06,9.57e-06,6.76e-07,1.02e-05,Italy,emilia-romagna,8
7,8,335.11,15.79,188.27,197.49,4.11e-06,4.31e-06,1.13e-05,7.94e-07,1.21e-05,Italy,emilia-romagna,8
8,9,326.77,15.35,192.16,201.72,3.73e-06,4.31e-06,1.02e-05,7.22e-07,1.10e-05,Italy,emilia-romagna,8
9,10,326.04,16.25,196.11,205.87,4.07e-06,4.31e-06,1.12e-05,7.88e-07,1.19e-05,Italy,emilia-romagna,8


## Update HADA

Now that we have the benchmark data comprehensive of the new metrics, we can use it within the HADA framework.

Before that, it is necessary to set a configuration file for each algorithm. That is nothing but a `.json` for each combination of algorithm and hardware platform containing informations about the algorithm, in particular:
* `HW_ID`: the id of hardware platform on which the algorithm was executed;
* `hyperparams`: the list of hyperparameters, i.e. the parameters controlling algorithm configuration. In case of ANTICIPATE and CONTINGENCY, we have just one parameter, namely the number of scenarios for ANTICIPATE and the number of traces for CONTINGENCY;
* `targets`: the list of targets, i.e. the metrics measured during the benchmark phase, with indications of the type of variables and eventually upper and lower bounds for them. These attributes will be the regression targets for the ML models trained by HADA.

For example, the following is the configuration file for ANTICIPATE.

In [7]:
config_file = "../data/configs/anticipate_mbp19.json"
config = json.load(open(config_file))
print(json.dumps(config, indent=2))

{
  "name": "anticipate",
  "HW_ID": "mbp19",
  "HW_price": null,
  "hyperparams": [
    {
      "ID": "nScenarios",
      "description": null,
      "type": "int",
      "LB": null,
      "UB": null
    }
  ],
  "targets": [
    {
      "ID": "time(sec)",
      "description": null,
      "LB": null,
      "UB": null
    },
    {
      "ID": "sol(keuro)",
      "description": null,
      "LB": null,
      "UB": null
    },
    {
      "ID": "memAvg(MB)",
      "description": null,
      "LB": null,
      "UB": null
    },
    {
      "ID": "CO2e(kg)",
      "description": null,
      "LB": null,
      "UB": null
    },
    {
      "ID": "CO2eRate(kg/s)",
      "description": null,
      "LB": null,
      "UB": null
    },
    {
      "ID": "cpuEnergy(kW)",
      "description": null,
      "LB": null,
      "UB": null
    },
    {
      "ID": "ramEnergy(kW)",
      "description": null,
      "LB": null,
      "UB": null
    },
    {
      "ID": "totEnergy(kW)",
      "description": null

The HADA framework will then train a set of Machine Learning models on the benchmark data. For both algorithm considered, a model will be created for each of the targets specified in the configuration file. The models are then embedded inside an optimization model as a set of constraints by [EMLlib](https://github.com/emlopt/emllib).

### Optimization request

Now, we can test the framework by submitting an optimization request in a json like format as the following, specifying an objective and a set of constraints

In [8]:
request = {
        "algorithm":"anticipate",
        "robustness_fact": None,
        "objective": {"target": "CO2e(kg)", "type": "min"},
        "constraints": [
            {'target': 'time(sec)', 'type': 'leq', 'value': 300},
            {'target': 'sol(keuro)', 'type': 'leq', 'value': 350},
            {'target': 'memAvg(MB)', 'type': 'leq', 'value': 350}
        ]
    }

This requires minimizing a specific target, in this case the CO2e emissions, while respecting constraints on the computation time ($\mathsf{time} \leq 300$), the solution cost ($\mathsf{sol} \leq 350$) and the average memory used ($\mathsf{memAvg} \leq 350$). By sending the request, we get the configuration of the algorithm that minimizes $\mathsf{CO_2e}$ emissions and satisfies the imposed constraints.

In [9]:
ret = util.optimize(request)
print(ret)

{'solution': {'hw': 'mbp19', 'nScenarios': 1, 'CO2e(kg)': 5.344659999999865e-06, 'memAvg(MB)': 344.46599999999995, 'time(sec)': 197.53200000000004, 'sol(keuro)': 268.16766666666666}}


### HADA UI

This is also what happens in the HADA web application. Here's what the user interface of HADA looks like after updating the code with the new targets

![HADA UI](assets/hada-ui.png)

By using the interface, we can easily send an optimisation request

![optreq](assets/hada-optimisation-request.png)

And obtain a solution:

![solution](assets/hada-solution.png)

## Final Considerations

In this work, the HADA framework was tested with the addional variables measured by Codecarbon for addressing the issue of Sustainable Hardware Dimensioning, finding the optimal algorithm and hardware configuration that minimizes the algorithm's Carbon Emissions. This has been a very interesting work that i decided to explore further, by choosing this as a project for my Thesis. Before concluding, there are just a few things i would like to point out about some problems that will be fixed in future work.

### Memory usage anomalies

when looking at data gathered executing the algorithms on Leonardo, i have noticed some strange values for the `memPeak(MB)` attribute, which was much lower than the average memory value `memPeak(MB)`

In [10]:
filename = 'anticipate_leonardo.csv'
benchmark_data = util.display_anomaly_data(filename)
display(benchmark_data)

,nScenarios,sol(keuro),time(sec),memAvg(MB),memPeak(MB)
0,1,364.16,2.59,116.96,0.12
1,2,355.45,2.35,158.84,0.16
2,3,354.44,3.26,195.47,0.19
3,4,353.03,4.23,229.82,0.23
4,5,352.39,5.06,274.03,0.27


This anomaly was not present in the data gathered on my machine, so i would have wanted to run new tests on Leonardo to understand the source of the problem. Unfortunately, this occurred in conjunction with the flooding in Emilia Romagna, which caused major problems for CINECA, resulting in the suspension of all HPC services until further notice. It was therefore not possible to replicate the tests on Leonardo. Of course, as soon as the HPC services will be restored, i will start working to solve this issue.

### Wrong values for `country` and `region` attributes

First thing is that codecarbon offers two main tracking modalities, online (with internet access) and offline (without internet access) tracking. For running the experiment presented in this notebook i used the online mode, which automatically detects the `country` and `region` attributes. However, this does not always select the right country. Indeed, some dataset entries have `CAN` and `quebec`:

In [11]:
filename = 'contingency_leonardo.csv'
benchmark_data = util.display_benchmark_data(filename)
display(benchmark_data)

,nTraces,sol(keuro),time(sec),memAvg(MB),memPeak(MB),CO2e(kg),CO2eRate(kg/s),cpuEnergy(kW),ramEnergy(kW),totEnergy(kW),country,region,cpuCount
0,1,369.19,11.14,265.88,0.37,3.07e-05,3.68e-07,2.71e-03,4.32e-03,1.29e-02,Canada,quebec,4
1,2,390.20,20.20,381.80,0.59,3.02e-05,3.67e-07,2.66e-03,4.25e-03,1.27e-02,Canada,quebec,4
2,3,374.38,31.73,484.35,0.80,3.05e-05,3.67e-07,2.69e-03,4.31e-03,1.28e-02,Canada,quebec,4
3,4,332.00,42.84,586.95,1.00,3.03e-05,3.68e-07,2.69e-03,4.27e-03,1.28e-02,Canada,quebec,4
4,5,333.78,54.21,684.13,1.20,3.02e-05,3.68e-07,2.68e-03,4.26e-03,1.27e-02,Canada,quebec,4
5,6,333.63,65.64,783.86,1.40,3.00e-05,3.68e-07,2.66e-03,4.24e-03,1.26e-02,Canada,quebec,4
6,7,341.48,77.27,881.06,1.60,3.03e-05,3.68e-07,2.69e-03,4.28e-03,1.28e-02,Canada,quebec,4
7,8,335.11,89.73,982.60,1.81,3.09e-05,3.68e-07,2.74e-03,4.35e-03,1.30e-02,Canada,quebec,4
8,9,326.77,101.01,1081.14,2.01,3.09e-05,3.68e-07,2.75e-03,4.36e-03,1.30e-02,Canada,quebec,4
9,10,326.04,113.24,1182.58,2.21,3.09e-05,3.68e-07,2.75e-03,4.36e-03,1.30e-02,Canada,quebec,4


This can alter the results, since the emissions are estimated based on the data about the *Energy Mix* for each country [[4]](https://github.com/mlco2/codecarbon/blob/master/codecarbon/data/private_infra/global_energy_mix.json). 

On the other hand, the offline tracker allows the developer to explicitly state the Country by setting the Country ISO code (e.g. ITA) as a parameter when initializing the tracker object. So, by using the offline tracker we can be sure that codecarbon will use the right Energy Mix data. Incorporating this aspects is one of the objectives of the thesis work, as well as including data about Energy Mix for each country directly in HADA, for enabling estimation of the emissions also in different countries without having to repeat the benchmark phase.